Implement CNN for classifying digits in MNIST dataset using PyTorch. Display the classification accuracy in the form of a Confusion matrix. Verify the number of learnable parameters in the model.

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [2]:
mnist_train=datasets.MNIST(root='./data',download=True,train=True,transform=ToTensor())
mnist_test=datasets.MNIST(root='./data',download=True,train=False,transform=ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 215490767.59it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 76668160.65it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 145840269.01it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 15191809.22it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
bsize=4
train=DataLoader(mnist_train,batch_size=bsize,shuffle=True)
test=DataLoader(mnist_test,batch_size=bsize,shuffle=True)

In [4]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.convLayer=nn.Sequential(nn.Conv2d(1,64,kernel_size=3),
                               nn.ReLU(),
                               nn.MaxPool2d((2,2),stride=2),
                               nn.Conv2d(64,128,kernel_size=3),
                               nn.ReLU(),
                               nn.MaxPool2d((2,2),stride=2),
                               nn.Conv2d(128,64,3),
                               nn.ReLU(),
                               nn.MaxPool2d((2,2),stride=2))
        self.classfn=nn.Sequential(nn.Linear(64,20,bias=True),
                                   nn.ReLU(),
                                   nn.Linear(20,10,bias=True))
    def forward(self,x):
        features=self.convLayer(x)
        return self.classfn(features.view(bsize,-1))

In [5]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model=CNNClassifier().to(device)
lossfn=nn.CrossEntropyLoss()
losslist=[]
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [7]:
print(model)
totalparams=0
for name,param in model.named_parameters():
    count=param.numel()
    totalparams+=count
print(totalparams)

CNNClassifier(
  (convLayer): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classfn): Sequential(
    (0): Linear(in_features=64, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
  )
)
149798


In [ ]:
epochs=5
for epoch in range(epochs):
    model.train(True)
    totalloss=0.0
    for i,data in enumerate(train):
        ip,op=data[0].to(device),data[1].to(device)
        optimizer.zero_grad()
        pred=model(ip)
        # _,predClass=torch.max(pred,dim=1)
        loss=lossfn(pred,op)
        loss.backward()
        optimizer.step()
        totalloss+=loss.item()
    avgLoss=totalloss/len(train)*bsize
    print(f"epoch={epoch}, loss={avgLoss}")

epoch=0, loss=1.8287496081486732
epoch=1, loss=0.3632481613320119
epoch=2, loss=0.24788857465341893
epoch=3, loss=0.1869555517185256


In [12]:
model.eval()

count=0
for _,testimage in enumerate(test):
    ip,op=testimage[0].to(device),testimage[1].to(device)
    count+=1
    if count==1: break

with torch.no_grad():
    predicted=model(ip)
    _,predictedOutput=torch.max(predicted,dim=1)

print(f"actual label={op}")
print(f"predicted label={predictedOutput}")

actual label=tensor([2, 6, 7, 6])
predicted label=tensor([2, 6, 7, 6])


In [11]:
correct=0
total=len(test)*bsize
count=0
for _,vdata in enumerate(test):
    vip,vop=vdata[0].to(device),vdata[1].to(device)
    vpred=model(vip)
    _,vclass=torch.max(vpred,dim=1)
    correct+=(vclass==vop).sum()
accuracy=(correct/total)*100
print(f"accuracy={accuracy}")

accuracy=98.43999481201172
